In [ ]:
%load_ext autoreload
%autoreload 2

from boolrank import DualSiglip2Model

# model = DualSiglip2Model('BAAI/llm-embedder')
model = DualSiglip2Model('BAAI/bge-small-en-v1.5')
# model = DualSiglip2Model('dmis-lab/biobert-v1.1')

In [2]:
from evaluation import evaluate_on_generated

evaluate_on_generated(model)

,model,spearman,avg_queries_per_prompt,f3_variance
0,HuggingfaceH4,0.1954,4.4615,0.0064
1,gpt-3.5-turbo-0125,0.2921,4.9000,0.0055
2,gpt-3.5-turbo-1106,0.1966,5.6250,0.0049
3,gpt-4-1106-preview,0.5975,3.3235,0.0090
4,gpt-4o-mini,0.4350,5.0500,0.0065
5,meta-llama,0.2317,5.5128,0.0085
6,mistralai,0.2555,4.2051,0.0076
7,o1-2024-12-17,-0.0692,4.4359,0.0048
8,open-mistral-7b,0.2850,4.3333,0.0071
9,open-mixtral-8x7b,0.1733,4.6923,0.0095


In [3]:
evaluate_on_generated(model, ["prompt_id", "id"])

c:\Users\Simon\Documents\Ordner\Uni\Master\25-Sommer\Research Project\project-doehl-boolean-clip\evaluation.py:202: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  avg_row[main_name] = "Average"


,id,spearman,avg_queries_per_prompt,f3_variance
0,1,0.2592,5.7143,0.0005
1,2,0.0918,4.0000,0.0003
2,3,-0.0364,6.7143,0.0187
3,4,0.1204,5.0000,0.0017
4,6,0.5429,6.1429,0.0106
5,7,-0.1313,6.2857,0.0017
6,8,-0.1639,5.1429,0.0005
7,10,-0.2864,7.0000,0.0137
8,11,-0.0663,5.5714,0.0106
9,12,-0.1440,4.8333,0.0131


In [4]:
model.load(r"models\clip\bge-small-en-v1.5\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# model.load(r"models\clip\biobert-v1.1\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-14110\model.safetensors")
evaluate_on_generated(model)

,model,spearman,avg_queries_per_prompt,f3_variance
0,HuggingfaceH4,0.0082,4.4615,0.0064
1,gpt-3.5-turbo-0125,-0.0011,4.9000,0.0055
2,gpt-3.5-turbo-1106,0.0761,5.6250,0.0049
3,gpt-4-1106-preview,0.1447,3.3235,0.0090
4,gpt-4o-mini,0.0052,5.0500,0.0065
5,meta-llama,0.0097,5.5128,0.0085
6,mistralai,-0.0231,4.2051,0.0076
7,o1-2024-12-17,-0.0744,4.4359,0.0048
8,open-mistral-7b,0.0273,4.3333,0.0071
9,open-mixtral-8x7b,-0.0079,4.6923,0.0095


In [5]:
evaluate_on_generated(model, ["prompt_id", "id"])

c:\Users\Simon\Documents\Ordner\Uni\Master\25-Sommer\Research Project\project-doehl-boolean-clip\evaluation.py:202: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  avg_row[main_name] = "Average"


,id,spearman,avg_queries_per_prompt,f3_variance
0,1,0.0388,5.7143,0.0005
1,2,0.2408,4.0000,0.0003
2,3,0.0238,6.7143,0.0187
3,4,-0.1888,5.0000,0.0017
4,6,0.2878,6.1429,0.0106
5,7,-0.2568,6.2857,0.0017
6,8,0.2701,5.1429,0.0005
7,10,-0.1901,7.0000,0.0137
8,11,-0.1173,5.5714,0.0106
9,12,-0.3857,4.8333,0.0131


In [6]:
model.load(r"models\clip\bge-small-en-v1.5\b16_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
evaluate_on_generated(model)

,model,spearman,avg_queries_per_prompt,f3_variance
0,HuggingfaceH4,0.1533,4.4615,0.0064
1,gpt-3.5-turbo-0125,0.2605,4.9000,0.0055
2,gpt-3.5-turbo-1106,0.1825,5.6250,0.0049
3,gpt-4-1106-preview,0.5613,3.3235,0.0090
4,gpt-4o-mini,0.3709,5.0500,0.0065
5,meta-llama,0.1335,5.5128,0.0085
6,mistralai,0.1005,4.2051,0.0076
7,o1-2024-12-17,-0.0487,4.4359,0.0048
8,open-mistral-7b,0.1811,4.3333,0.0071
9,open-mixtral-8x7b,0.1280,4.6923,0.0095


In [7]:
evaluate_on_generated(model, ["prompt_id", "id"])

c:\Users\Simon\Documents\Ordner\Uni\Master\25-Sommer\Research Project\project-doehl-boolean-clip\evaluation.py:202: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  avg_row[main_name] = "Average"


,id,spearman,avg_queries_per_prompt,f3_variance
0,1,0.3000,5.7143,0.0005
1,2,0.1204,4.0000,0.0003
2,3,-0.0197,6.7143,0.0187
3,4,0.0571,5.0000,0.0017
4,6,0.4901,6.1429,0.0106
5,7,-0.2197,6.2857,0.0017
6,8,-0.2112,5.1429,0.0005
7,10,-0.1592,7.0000,0.0137
8,11,-0.1776,5.5714,0.0106
9,12,-0.0476,4.8333,0.0131
